In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("/kaggle/input/mqttset/train70_augmented.csv")
test = pd.read_csv("/kaggle/input/mqttset/test30_augmented.csv")

In [3]:
# tamanho dos df
print("Train shape: ", train.shape)
print("Test shape: ", test.shape, "\n")

print("Train target:\n", train['target'].value_counts())
print("Test target:\n",test['target'].value_counts())

Train shape:  (14000000, 34)
Test shape:  (6000000, 34) 

Train target:
 target
legitimate    7000000
bruteforce    1400000
flood         1400000
slowite       1400000
dos           1400000
malformed     1400000
Name: count, dtype: int64
Test target:
 target
legitimate    3000000
dos            600000
slowite        600000
flood          600000
bruteforce     600000
malformed      600000
Name: count, dtype: int64


In [4]:
print(train.keys())

Index(['tcp.flags', 'tcp.time_delta', 'tcp.len', 'mqtt.conack.flags',
       'mqtt.conack.flags.reserved', 'mqtt.conack.flags.sp', 'mqtt.conack.val',
       'mqtt.conflag.cleansess', 'mqtt.conflag.passwd', 'mqtt.conflag.qos',
       'mqtt.conflag.reserved', 'mqtt.conflag.retain', 'mqtt.conflag.uname',
       'mqtt.conflag.willflag', 'mqtt.conflags', 'mqtt.dupflag',
       'mqtt.hdrflags', 'mqtt.kalive', 'mqtt.len', 'mqtt.msg', 'mqtt.msgid',
       'mqtt.msgtype', 'mqtt.proto_len', 'mqtt.protoname', 'mqtt.qos',
       'mqtt.retain', 'mqtt.sub.qos', 'mqtt.suback.qos', 'mqtt.ver',
       'mqtt.willmsg', 'mqtt.willmsg_len', 'mqtt.willtopic',
       'mqtt.willtopic_len', 'target'],
      dtype='object')


In [5]:
# divisão entre tráfego legítimo e ataques de negação de serviço

# ----------------------------------------------------------------

# conj de treinamento

bruteforce = []
malformed = []
slowite = []

for i in train.index:

    if train['target'][i] == 'bruteforce':
        bruteforce.append(i)
        
    if train['target'][i] == 'malformed':
        malformed.append(i)
        
    if train['target'][i] == 'slowite':
        slowite.append(i)

# exclusao das linhas
train = train.drop(malformed)
train = train.drop(slowite)
train = train.drop(bruteforce)

# -------------------------------------------------------------------------

# conj de teste

bruteforce = []
malformed = []
slowite = []

for i in test.index:

    if test['target'][i] == 'bruteforce':
        bruteforce.append(i)
        
    if test['target'][i] == 'malformed':
        malformed.append(i)
        
    if test['target'][i] == 'slowite':
        slowite.append(i)
        
test = test.drop(bruteforce)
test = test.drop(malformed)
test = test.drop(slowite)

# -----------------------------------------------------------------------------------

# novo tamanho df
print("Train: ")
print(train['target'].value_counts())
print("\n")
print("Test: ")
print(test['target'].value_counts())

Train: 
target
legitimate    7000000
flood         1400000
dos           1400000
Name: count, dtype: int64


Test: 
target
legitimate    3000000
dos            600000
flood          600000
Name: count, dtype: int64


In [6]:
# lógica para deixar df com duas classes 
# legitimo e malicioso

for i in train.index:
    if train['target'][i] != 'legitimate':
        train.at[i, 'target'] = 'malicioso'
         
for i in test.index:
    if test['target'][i] != 'legitimate':
        test.at[i, 'target'] = 'malicioso'
        
# novo dataframe
print(train['target'].value_counts())
print(test['target'].value_counts())

target
legitimate    7000000
malicioso     2800000
Name: count, dtype: int64
target
legitimate    3000000
malicioso     1200000
Name: count, dtype: int64


In [7]:
# lógica de balanceamento 
# deixa a qtdade igual para as duas classes

legitimate_train = []
legitimate_test = []

for i in train.index:
    if train['target'][i] == 'legitimate':
        legitimate_train.append(i)
        
for i in test.index:
    if test['target'][i] == 'legitimate':
        legitimate_test.append(i)
        
        
# para balancear: 7.000.000 - 2.800.000 = 4.200.000 
# isso exclui as linhas, deixando as classes com 2.800.000 linhas cada
legitimate_train = legitimate_train[:4200000]
train = train.drop(legitimate_train)

# 3.000.000 - 1.200.000 = 1.800.000
# cada classe com 1.800.000 linhas
legitimate_test = legitimate_test[:1800000]
test = test.drop(legitimate_test)

# novo tamanho
print(train['target'].value_counts())
print(test['target'].value_counts())

target
malicioso     2800000
legitimate    2800000
Name: count, dtype: int64
target
malicioso     1200000
legitimate    1200000
Name: count, dtype: int64


In [8]:
# para carregar os dados sem precisar processar novamente
train.to_pickle("train_processed.pkl")
test.to_pickle("test_processed.pkl")